In [3]:
import os
import requests
import pandas as pd
import numpy as np
import json
import time
import random
from lxml import etree

### 影信息通过动态加载，所有的信息都藏在基础网页，唯一变动的是start

In [5]:
url1 = 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=0'
url2 = 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=20'
url3 = 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=40'
url4 = 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=60'

### 构造爬取的网址

In [9]:
#构造网页
def format_url(num):
    urls = []
    base_url = 'https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start={}'
    for i in range(0,20 * num,20):
        url = base_url.format(i)
        urls.append(url)
    return urls

#这里是爬取10页，可以自行更改参数
urls = format_url(10)

### 伪装请求头

In [8]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

###  解析单页

In [11]:
def parse_base_info(url,headers):
    html = requests.get(url,headers = headers)   
    bs = json.loads(html.text)
    df = pd.DataFrame()
    for i in bs['data']:
        casts = i['casts']  #主演
        cover = i['cover']  #海报
        directors = i['directors']  #导演
        m_id = i['id']  #ID
        rate = i['rate'] #评分
        star = i['star'] #标记人数 
        title = i['title']  #片名
        url = i['url']  #网址
        cache = pd.DataFrame({'主演':[casts],'海报':[cover],'导演':[directors],
                              'ID':[m_id],'评分':[rate],'标记':[star],'片名':[title],'网址':[url]})
        df = pd.concat([df,cache])
    return df

### 循环批量爬取电影

In [12]:
result = pd.DataFrame()

count = 1
for url in urls:
    df = parse_base_info(url,headers = headers)
    result = pd.concat([result,df])
    time.sleep(random.random() + 2)
    print('I had crawled page of:%d' % count)
    count += 1

I had crawled page of:1
I had crawled page of:2
I had crawled page of:3
I had crawled page of:4
I had crawled page of:5
I had crawled page of:6
I had crawled page of:7
I had crawled page of:8
I had crawled page of:9
I had crawled page of:10


In [13]:
result.head()

,ID,主演,导演,标记,海报,片名,网址,评分
0,26752088,"[徐峥, 王传君, 周一围, 谭卓, 章宇]",[文牧野],45,https://img3.doubanio.com/view/photo/s_ratio_p...,我不是药神,https://movie.douban.com/subject/26752088/,9.0
0,1295644,"[让·雷诺, 娜塔莉·波特曼, 加里·奥德曼, 丹尼·爱罗, 彼得·阿佩尔]",[吕克·贝松],45,https://img3.doubanio.com/view/photo/s_ratio_p...,这个杀手不太冷,https://movie.douban.com/subject/1295644/,9.4
0,1292052,"[蒂姆·罗宾斯, 摩根·弗里曼, 鲍勃·冈顿, 威廉姆·赛德勒, 克兰西·布朗]",[弗兰克·德拉邦特],50,https://img3.doubanio.com/view/photo/s_ratio_p...,肖申克的救赎,https://movie.douban.com/subject/1292052/,9.7
0,26266893,"[屈楚萧, 吴京, 李光洁, 吴孟达, 赵今麦]",[郭帆],40,https://img3.doubanio.com/view/photo/s_ratio_p...,流浪地球,https://movie.douban.com/subject/26266893/,7.9
0,1292720,"[汤姆·汉克斯, 罗宾·怀特, 加里·西尼斯, 麦凯尔泰·威廉逊, 莎莉·菲尔德]",[罗伯特·泽米吉斯],45,https://img3.doubanio.com/view/photo/s_ratio_p...,阿甘正传,https://movie.douban.com/subject/1292720/,9.4


### 解析单个页面，获取详细的电影信息

In [14]:
def parse_movie_info(url,headers = headers,ip = ''):
    if ip == '':
        html = requests.get(url,headers = headers)
    else:
        html = requests.get(url,headers = headers,proxies = ip)
    bs = etree.HTML(html.text)
    #片名
    title = bs.xpath('//div[@id = "wrapper"]/div/h1/span')[0].text  
    #上映时间
    year = bs.xpath('//div[@id = "wrapper"]/div/h1/span')[1].text   
    #电影类型
    m_type = []
    for t in bs.xpath('//span[@property = "v:genre"]'):
        m_type.append(t.text)   
    a = bs.xpath('//div[@id= "info"]')[0].xpath('string()')
    #片长
    m_time =a[a.find('片长: ') + 4:a.find('分钟\n')]  #时长
    #地区
    area = a[a.find('制片国家/地区:') + 9:a.find('\n        语言')]  #地区
    #评分人数
    try:
        people = bs.xpath('//a[@class = "rating_people"]/span')[0].text
    #评分分布
        rating = {}
        rate_count = bs.xpath('//div[@class = "ratings-on-weight"]/div')
        for rate in rate_count:
            rating[rate.xpath('span/@title')[0]] = rate.xpath('span[@class = "rating_per"]')[0].text
    except:
        people = 'None'
        rating = {}
    #简介
    try:
        brief = bs.xpath('//span[@property = "v:summary"]')[0].text.strip('\n                                \u3000\u3000')
    except:
        brief = 'None'
    try:
        hot_comment = bs.xpath('//div[@id = "hot-comments"]/div/div/p/span')[0].text
    except:
        hot_comment = 'None'
    cache = pd.DataFrame({'片名':[title],'上映时间':[year],'电影类型':[m_type],'片长':[m_time],
                          '地区':[area],'评分人数':[people],'评分分布':[rating],'简介':[brief],'热评':[hot_comment],'网址':[url]})
    return cache

### 批量访问单个电影页面

In [15]:
movie_result = pd.DataFrame()
ip = ''  #这里构建自己的IP池
count2 = 1
cw = 1

for url,name in zip(result['网址'].values,result['片名'].values):
#for name,url in wrongs.items():
    try:
        cache = parse_movie_info(url,headers = headers,ip = ip)
        movie_result = pd.concat([movie_result,cache])
        #time.sleep(random.random())
        print('我们爬取了第:%d部电影-------%s' % (count2,name))
        count2 += 1
    except:
        print('滴滴滴滴滴，第{}次报错'.format(cw))
        print('ip is:{}'.format(ip))
        cw += 1
        time.sleep(2)
        continue

我们爬取了第:1部电影-------我不是药神
我们爬取了第:2部电影-------这个杀手不太冷
我们爬取了第:3部电影-------肖申克的救赎
我们爬取了第:4部电影-------流浪地球
我们爬取了第:5部电影-------阿甘正传
我们爬取了第:6部电影-------复仇者联盟3：无限战争
我们爬取了第:7部电影-------盗梦空间
我们爬取了第:8部电影-------西虹市首富
我们爬取了第:9部电影-------泰坦尼克号
我们爬取了第:10部电影-------千与千寻
我们爬取了第:11部电影-------霸王别姬
我们爬取了第:12部电影-------三傻大闹宝莱坞
我们爬取了第:13部电影-------让子弹飞
我们爬取了第:14部电影-------怦然心动
我们爬取了第:15部电影-------摔跤吧！爸爸
我们爬取了第:16部电影-------毒液：致命守护者
我们爬取了第:17部电影-------疯狂动物城
我们爬取了第:18部电影-------忠犬八公的故事
我们爬取了第:19部电影-------一出好戏
我们爬取了第:20部电影-------当幸福来敲门
我们爬取了第:21部电影-------海上钢琴师
我们爬取了第:22部电影-------大话西游之大圣娶亲
我们爬取了第:23部电影-------海王
我们爬取了第:24部电影-------楚门的世界
我们爬取了第:25部电影-------你的名字。
我们爬取了第:26部电影-------阿凡达
我们爬取了第:27部电影-------少年派的奇幻漂流
我们爬取了第:28部电影-------星际穿越
我们爬取了第:29部电影-------头号玩家
我们爬取了第:30部电影-------无双
我们爬取了第:31部电影-------放牛班的春天
我们爬取了第:32部电影-------飞屋环游记
我们爬取了第:33部电影-------机器人总动员
我们爬取了第:34部电影-------那些年，我们一起追的女孩
我们爬取了第:35部电影-------龙猫
我们爬取了第:36部电影-------寻梦环游记
我们爬取了第:37部电影-------红海行动
我们爬取了第:38部电影-------初恋这件小事
我们爬取了第:39部电影-------大话西游之月光宝盒
我们爬取了第:40部电影-----

In [17]:
movie_result.head()

,上映时间,地区,热评,片名,片长,电影类型,简介,网址,评分人数,评分分布
0,(2018),中国大陆,“你敢保证你一辈子不得病？”纯粹、直接、有力！常常感叹：电影只能是电影。但每看到这样的佳作，...,我不是药神,117,"[剧情, 喜剧]",普通中年男子程勇（徐峥 饰）经营着一家保健品店，失意又失婚。不速之客吕受益（王传君 饰）的到...,https://movie.douban.com/subject/26752088/,1174897,"{'还行': '7.0%', '力荐': '57.4%', '较差': '0.5%', '很..."
0,(1994),法国,他们居然没做爱,这个杀手不太冷 Léon,110分钟(剧场版) / 133分钟(国际版)\n 又名: 杀手莱昂 / 终极...,"[剧情, 动作, 犯罪]",里昂（让·雷诺饰）是名孤独的职业杀手，受人雇佣。一天，邻居家小姑娘马蒂尔达（纳塔丽·波特曼饰...,https://movie.douban.com/subject/1295644/,1380628,"{'还行': '3.2%', '力荐': '74.2%', '较差': '0.2%', '很..."
0,(1994),美国,关于希望最强有力的注释。,肖申克的救赎 The Shawshank Redemption,142,"[剧情, 犯罪]",20世纪40年代末，小有成就的青年银行家安迪（蒂姆·罗宾斯 Tim Robbins 饰）因涉...,https://movie.douban.com/subject/1292052/,1525345,"{'还行': '1.5%', '力荐': '84.6%', '较差': '0.1%', '很..."
0,(2019),中国大陆,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,流浪地球,125,"[科幻, 灾难]",近未来，科学家们发现太阳急速衰老膨胀，短时间内包括地球在内的整个太阳系都将被太阳所吞没。为了...,https://movie.douban.com/subject/26266893/,1264654,"{'还行': '22.0%', '力荐': '33.1%', '较差': '4.7%', '..."
0,(1994),美国,我生命里最温暖的一部电影,阿甘正传 Forrest Gump,142,"[剧情, 爱情]",阿甘（汤姆·汉克斯 饰）于二战结束后不久出生在美国南方阿拉巴马州一个闭塞的小镇，他先天弱智，...,https://movie.douban.com/subject/1292720/,1192711,"{'还行': '2.9%', '力荐': '76.0%', '较差': '0.2%', '很..."


### 文件存储

In [18]:
result.to_excel('电影基本信息大全.xlsx')
movie_result.to_excel('电影详细信息.xlsx')